In [1]:
#!/usr/bin/env python
import os 
import pandas as pd
from wmf import wmf
import numpy as np 
import glob 
import pylab as pl
import json
import MySQLdb
import csv
import matplotlib
import matplotlib.font_manager
import datetime as dt
from datetime import timedelta
import datetime as dt
import pickle
import matplotlib.dates as mdates
import netCDF4
import textwrap
from multiprocessing import Pool
import alarmas as al

import warnings
warnings.filterwarnings('ignore')

In [2]:
def Graph_Levels(ruta_inQhist,ruta_inQsim,ruta_outLevelspng,ruta_out_rain,date,nodosim,codeest,mediah,ruta_outNsim,verbose=True):
    ''' #Genera graficas para cada nodo en .png comparando Nsims y Nobs y los niveles de riesgo, calculando criterio de calibracion
        Nash-Sutcliffe. Se obtiene Nsim a partir de Qsim, restando la media de Qsim y sumando la media historica de Qobs 'mediah'.
        #Funcion operacional.
        #Argumentos:
        ruta_inQhist: string, ruta de donde se lee el Qhist simulado de una parametrizacion. 
        ruta_inQsim: string, ruta de donde se lee el Qsim de una parametrizacion.
        ruta_outLevelspng: string, ruta de la carpeta donde se guardan los .pngs.
        ruta_out_rain: string, ruta de la carpeta de donde se leen los .hdr de los campos de radar de esa cuenca.
        date: string, con la fecha y hora en la que se corre la imagen, esto para el nombre del .png.
        nodosim: lista de int's de los nodos que corresponden con estaciones de nivel para comparar simulado.
        codeest: lista de int's de los codigos de estaciones que corresponden con nodos de simulacion para comparar simulado.
        mediah: lista de int's de las medias historicas de Nobs en las estaciones que corresponden con nodos de simulacion.
        ruta_outNsim: string, ruta donde de la carpeta donde se guardan las series Nsim corregidas con Nobs.
        verbose: boolean, condicional para que devuelva los prints de la ejecucion. Default= True.
        #Nota: nodosim, codeest y mediah deben tener el mismo size ya que cada pos hace referencia a la misma estacion.
    '''
    #Se cambia formato de 
    nodo_ests=np.array([int(nodo) for nodo in nodosim.split(',')])
    code_ests=np.array([int(nodo) for nodo in codeest.split(',')])
    media_hs=np.array([float(nodo) for nodo in mediah.split(',')])

    #Se leen resultados de simulacion de todas las par. para todos los nodos.
    #Leer ultima hora de historico Qsim para cada par.
    rutah=ruta_inQhist+'*.json'
    readh=glob.glob(rutah)
    #Leer las simulacion actual+extrapolacion
    ruta1=ruta_inQsim+'*.json'
    read1=glob.glob(ruta1)
    #Guarda series completas e hist para sacar Nash
    Qhist=[];Qact=[]
    for rqhist,rqsim in zip(np.sort(readh),np.sort(read1)):
        #Q HIST
        dfhist=pd.read_json(rqhist)
        # #crea index para poner nans en faltantes, si los hay.
        # rngindex=pd.date_range(dfhist.index[0],dfhist.index[-1],freq='5min')
        # dfhist=dfhist.reindex(rngindex)
        #ultima hora, 12 pasos de 5 min.
        qhist=dfhist[-12:]
        #hist para sacar Nash, ultima hr del nodo de salida.
        Qhist.append(qhist[nodo_ests[0]][-12:])
        #Q ACT
        dfsim=pd.read_json(rqsim)
        qEst=dfsim
        #ult hr+ extrapolacion
        Qact.append(qhist.append(qEst))

    for nodo,code,media in zip(nodo_ests,code_ests,media_hs):
        #Lee ruta del archivo a guardar, si no existe se crea
        ruta_folder = ruta_outLevelspng+'/'+str(nodo)+'/'
        Esta = glob.glob(ruta_folder)
        if len(Esta) == 0:
            os.system('mkdir '+ruta_folder)
        ruta_out_png = ruta_folder+'LevelsSimNodo'+str(nodo)+'_'+date+'.png' 

        #series
        otra = glob.glob(ruta_outNsim)
        if len(otra) == 0:
            os.system('mkdir '+ruta_outNsim)
        #Obtiene las ruta de archivo de salida
        ruta_out_serie = ruta_outNsim+'NSim'+str(code)+'.csv'

        otra = glob.glob(ruta_outNsim)
        if len(otra) == 0:
            os.system('mkdir '+ruta_outNsim)

        #-------------------------------------------------------------------------------------------------------
        #Grafica comparativa de niveles, con escala de colores y backgroud de siata.
        #------------------------------------------------------------------------------------------------------
        fig= pl.figure(figsize=(12,9))
        ax= fig.add_subplot(111)    

        #Grafica de niveles simulados.
        #Colormap
        #-------------------------------------------------------------------------------------------------------
        parameters = np.linspace(0,len(Qact),len(Qact))
        # norm is a class which, when called, can normalize data into the [0.0, 1.0] interval.
        norm = matplotlib.colors.Normalize(
            vmin=np.min(parameters),
            vmax=np.max(parameters))
        #choose a colormap
        c_m =pl.cm.Spectral#nipy_spectral#winter#autumn#summer#PuBuGn
        # create a ScalarMappable and initialize a data structure
        s_m = pl.cm.ScalarMappable(cmap=c_m, norm=norm)
        s_m.set_array([])
        #------------------------------------------------------------------------------------------------------

    #     # Si el nodo tiene estacion instalada
    #     if  nodo in nodo_est:

        #Lluvia
        ruta_hdrp1=ruta_out_rain + 'Lluvia_historica.hdr'
        ruta_hdrp2=ruta_out_rain + 'Lluvia_actual.hdr'
        Phist=wmf.read_mean_rain(ruta_hdrp1,100000000000,0)
        Pextrapol=wmf.read_mean_rain(ruta_hdrp2,100000000000,0) 

        #-------------------------------------------------------------------------------------------------------
        #Consulta a base de datos: Nobs y Ns de alerta'
        #-------------------------------------------------------------------------------------------------------
        #Se usa las fechas de una serie sim para consultar en bd.
        serieN=Qact[0]
        FI=serieN.index.strftime('%Y-%m-%d')[0]
        FF=serieN.index.strftime('%Y-%m-%d')[-1]
        HI=serieN.index[0].strftime('%H:%M')
        HF=serieN.index[-1].strftime('%H:%M')
        # coneccion a bd con usuario operacional
        host   = '192.168.1.74'
        user   = 'siata_Oper'
        passwd = 'si@t@64512_operacional'
        bd     = 'siata'
        #Consulta a tabla estaciones
        Estaciones="SELECT Codigo,Nombreestacion, offsetN,N,action_level,minor_flooding,moderate_flooding,major_flooding  FROM estaciones WHERE codigo=("+str(code)+")"
        dbconn = MySQLdb.connect(host, user,passwd,bd)
        db_cursor = dbconn.cursor()
        db_cursor.execute(Estaciones)
        result = np.array(db_cursor.fetchall())
        #definicion de niveles de alerta y demas.
        nombreest=result[0][1] 
        n1=float(result[0][4])
        n2=float(result[0][5])
        n3=float(result[0][6])
        n4=float(result[0][7])
        #definicion de tipo N para consultar campo.
        tipo=int(result[0][3])
        if tipo == 1:#radar
            niv='ni'
        elif tipo == 0:#ultrasonido
            niv='pr'
        #Consulta a tabla datos
        sql_datos ="SELECT DATE_FORMAT(fecha,'%Y-%m-%d'), DATE_FORMAT(hora, '%H:%i:%s'), (" +str(result[0][2])+"-"+niv+"), calidad FROM datos WHERE cliente = ("+str(code)+") and fecha between '"+FI+"' and '"+FF+"' and hora between '"+HI+"' and '"+HF+"'"
        dbconn = MySQLdb.connect(host, user,passwd,bd)
        db_cursor = dbconn.cursor()
        db_cursor.execute(sql_datos)
        result_data = np.array(db_cursor.fetchall())
        data = pd.DataFrame(result_data)

        #Se organizan consulta en serie de tiempo.

        fe=[data[0][i]+'-'+data[1][i] for i in range(len(data))]; fe=np.array(fe)
        nobs=[float(data[2][i]) for i in range(len(data))];nobs=np.array(nobs)
        calidad=[int(data[3][i]) for i in range(len(data))];calidad=np.array(calidad)
        #se encuentran y eliminan los datos con datetime malos.
        badpos=[];dates=[]
        for i,date in enumerate(fe):
            try:
                dates.append(dt.datetime.strptime(date,'%Y-%m-%d-%H:%M:%S'))
            except:
                badpos.append(i)
        nobs=np.delete(nobs,badpos)
        calidad=np.delete(calidad,badpos)
        # serie
        Nobs=pd.Series(nobs,index=dates)

        #Se corrgie Nobs por calidad
        try:
            Nobs[np.where((calidad!=1)&(calidad!=2))[0]]=np.nan
        except:
            pass
        # Nobs[Nobs>float(offset)]=np.nan
        Nobs[Nobs>600.0]=np.nan

        #Calidad
        grad=50
        for k in range(1,(len(Nobs)-1)):
            d= Nobs[k]
            c= Nobs[k-1]
            e= Nobs[k+1]
            if abs(d-c)>grad and abs(d-e)>grad:
                Nobs[k]=np.nan

        #Poner Qsims en magnitud de Nobservados.
        Nnodo=[]
        for sim in Qact:
            serie_nodo=sim[nodo]
            Nsim=serie_nodo-serie_nodo.mean()+media
            Nnodo.append(Nsim)
        # Se guardan los Nsim para el programa de Mario
        Nsims=pd.DataFrame(np.array(Nnodo).T, index=Nnodo[0].index)
        Nsims.to_csv(ruta_out_serie)

        # Cosas para graficar niveles de riesgo.
        ylim4=n4+(n4*0.2)
        levels=[n1,n2,n3,n4,ylim4]
        lnames=['N1','N2', 'N3', 'N4']
        lcolors=['g','orange','orangered','indigo']

        #plot
        for i in range(0,len(levels)):
            try:
                ax.fill_between(x=[Nobs.index[0],serieN.index[-1]], 
                                y1=[levels[i],levels[i]],
                                y2=[levels[i+1],levels[i+1]], 
                                color = lcolors[i], 
                                alpha = 0.22,
                                label=lnames[i])
            except:
                pass


        #PLOT
        for i,parameter in zip(np.arange(0,len(Nnodo)),parameters):
            #NASH
            nash=wmf.__eval_nash__(Nobs,Nnodo[i])
            ax.plot(Nnodo[i],lw=2.5,linestyle='--', label='P0'+str(i+1)+'- NS:%.2f'%(nash),color=s_m.to_rgba(parameter))
        #Text ans ticks color.
        backcolor='dimgray'  
        #Obs
        ax.plot(Nobs,c='k',lw=3.5, label='Nobs')
        ax.set_title('Est. %s. %s ___ Fecha: %s'%(code,nombreest,serieN.index.strftime('%Y-%m-%d')[0]), fontsize=17,color=backcolor)
        ax.set_ylabel('Nivel  $[cm]$', fontsize=17,color=backcolor)
        ax.axvline(x=Nobs.index[-1],lw=2,color='gray',label='Now')

        # Second axis
        #plot
        axAX=pl.gca()
        ax2=ax.twinx()
        ax2AX=pl.gca()
        try:
            #Mean rainfall
            # Busca en Qact la primera pos del obs.
            p1=Phist[Phist.index.get_loc(Qact[0].index[0]):]
            # Busca en Pextrapol la ultima  pos del Pobs
            p2=Pextrapol[Pextrapol.index.get_loc(Phist.index[-1])+1:]
            P=p1.append(p2)
            #Se pasa a mm/h
            P=P*12.0

            #resto del plot
            ax2.fill_between(P.index,0,P,alpha=0.25,color='dodgerblue',lw=0)
            #limites
            ax2AX.set_ylim((0,20)[::-1]) 
        except:
            print 'Aviso: No se grafica Lluvia promedio, no exiten campos para la fecha en el historico'

        #Formato  resto de grafica.
        ax.tick_params(labelsize=14)
        ax.grid()
        ax.autoscale(enable=True, axis='both', tight=True)
        #setting default color of ticks and edges
        ax.tick_params(color=backcolor, labelcolor=backcolor)
        for spine in ax.spines.values():
            spine.set_edgecolor('gray')
        #color of legend text
        leg = ax.legend(ncol=3,loc=(0.26,-0.255),fontsize=12)
        for text in leg.get_texts():
            pl.setp(text, color = 'dimgray')

        #ylim para la grafica respecto a Nobs.
        ylim=n4+(n4*0.05)
        y_lim=Nobs.mean()*0.5
        ax.set_ylim(y_lim,ylim)

        #Formato resto de grafica - Second axis
        ax2.set_ylabel(u'Precipitacion media - cuenca [$mm$]',size=17,color=backcolor)    
        ax2.tick_params(labelsize=14)
        ax2.tick_params(color=backcolor, labelcolor=backcolor)
        for spine in ax2.spines.values():
            spine.set_edgecolor('gray')
        #Se guarda la figura.
        ax.figure.savefig(ruta_out_png,bbox_inches='tight')

    if verbose:
        print 'Aviso: Plot de niveles generado en '+ruta_out_png

In [3]:
ruta_configuracion_1 = '/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/configfile.md'
RutasList = al.get_rutesList(ruta_configuracion_1)

# Lee rutas de objetos de entrada
ruta_cuenca = al.get_ruta(RutasList, 'ruta_cuenca')


In [4]:
date=dt.datetime.now().strftime('%Y-%m-%d-%H:%M')
cuenca = wmf.SimuBasin(rute = ruta_cuenca, SimSlides = True)
# DictStore = al.get_modelConfig_lines(RutasList, '-s', 'Store')
ruta_out_rain = al.get_ruta(RutasList, 'ruta_rain')


#Se define ruta de donde se leeran los resultados a plotear
ruta_inQhist = al.get_ruta(RutasList,'ruta_qsim_hist')
ruta_inQsim = al.get_ruta(RutasList,'ruta_qsim')
#se leen cosas necesarias para la funcion
nodosim = al.get_ruta(RutasList,'nodosim')
codeest = al.get_ruta(RutasList,'codeestN')
mediah = al.get_ruta(RutasList,'mediaN')
#Lectura de rutas de salida de la imagen
ruta_outLevelspng = al.get_ruta(RutasList,'ruta_levelspng')
#Lectura de rutas donde guardar Nsim.
ruta_outNsim = al.get_ruta(RutasList,'ruta_niveles')
# verbose=True

In [5]:
Graph_Levels(ruta_inQhist,ruta_inQsim,ruta_outLevelspng,ruta_out_rain,date,nodosim,codeest,mediah,ruta_outNsim)

IndexError: index -1 is out of bounds for axis 0 with size 0